# Markov Decission Process

## Frozen Lake

In [1]:
import gym
import numpy as np

In [2]:
env = gym.make('FrozenLake-v0')

### Value based approach

Initialize $V(s)$ to arbitrary values <br>
Repeat<br>
&emsp; For all $s \in S$ <br>
&emsp; &emsp; For all $a \in A$ <br>
&emsp; &emsp; &emsp;  $Q(s,a) \gets E[r|s,a] + \gamma\sum_{s'\in S}P(s'|s,a)V(s')$ <br>
&emsp; &emsp; $V(s) \gets max_aQ(s,a)$ <br>
Until $V(s)$ converge


In [3]:
def value_iteration(env, gamma=1.0, n_iter=100000, threshold=1e-20):
    value = np.zeros(env.env.nS)
    policy = np.zeros(env.env.nS)
    for i in range(n_iter):
        prev_value = np.copy(value)
        Q_table = np.zeros((env.env.nS, env.env.nA))
        for state in range(env.env.nS):
            for action in range(env.env.nA):
                Q_table[state, action] = np.sum([
                    trans_prob * (reward_prob + gamma * prev_value[next_state])
                    for trans_prob, next_state, reward_prob, _ in env.env.P[state][action]
                ])
                
            value[state] = max(Q_table[state,:])
            policy[state] = np.argmax(Q_table[state,:])
        if np.sum(np.fabs(value-prev_value))<=threshold:
            print('Funkcija vrednosti je konvergirala nakon {} iteracija'.format(i+1))
            break
    return value, policy

In [4]:
optimal_value_function = value_iteration(env, gamma=1)
print(optimal_value_function[1])

Funkcija vrednosti je konvergirala nakon 1373 iteracija
[0. 3. 3. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]


### Policy based approach
Initialize a policy $\pi'$ to arbitrary values <br>
Repeat <br>
&emsp; $\pi \gets \pi'$ <br>
&emsp; Compute the values using $\pi$ by solving the linear equations:<br>
&emsp; $V^\pi(s) = E[r|s,\pi(s)] + \gamma\sum_{s'\in S}P(s'|s,\pi(s))V^\pi(s')$ <br>
&emsp; Improve the policy at each state: <br>
&emsp; $\pi'(s) \gets argmax_a(E[r|s,a] + \gamma\sum_{s' \in S}P(s'|s,a)V^\pi(s'))$ <br>
Until $\pi = \pi'$

In [5]:
def compute_value_function(env, policy, gamma=1.0, threshold=1e-10):
    value = np.zeros(env.env.nS)
    condition = True
    while condition:
        value_copy = np.copy(value)
        for state in range(env.env.nS):
            action = policy[state]
            value[state] = sum([
                trans_prob * (reward_prob + gamma * value_copy[next_state])
                for trans_prob, next_state, reward_prob, _ in env.env.P[state][action]
            ])
        condition =  np.sum(np.fabs(value_copy - value))>threshold
    return value  

In [6]:
def extract_policy(env, value, gamma=1.0):
    policy = np.zeros(env.env.nS)
    for state in range(env.env.nS):
        Q_table = []
        for action in range(env.env.nA):
            Q_table.append(np.sum([
                trans_prob * (reward_prob + gamma * value[next_state])
                for trans_prob, next_state, reward_prob, _ in env.env.P[state][action]
            ]))
        policy[state] = np.argmax(Q_table)
    return policy

In [7]:
def policy_iteration(env, n_iter=20000, gamma=1.0):
    policy = np.zeros(env.env.nS)
    for i in range(n_iter):
        new_value_function = compute_value_function(env, policy, gamma)
        new_policy = extract_policy(env, new_value_function, gamma)
        if (np.all(new_policy==policy)):
            print('Funkcija politika je konvergirala u {} koraku'.format(i+1))
            break
        policy = new_policy
    return new_policy

In [8]:
optimal_policy = policy_iteration(env)
print(optimal_policy)

Funkcija politika je konvergirala u 7 koraku
[0. 3. 3. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]
